# **Trajectory** on the reconstructed lane

**Trajectory computation**

In [17]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import csv

Loading files paths

In [18]:
PROJECT_ROOT = Path().resolve().parent.parent
INPUT_VIDEO_PATH = str(PROJECT_ROOT / "data" / "recording_2" / "Recording_2_normal_speed.mp4")
CSV_FILE_PATH = str(PROJECT_ROOT / "data" / "auxiliary_data" / "reconstructed_positions" / "Transformed_positions_2.csv")
OUTPUT_VIDEO_PATH = str(PROJECT_ROOT / "data" / "recording_2" / "Reconstructed_trajectory.mp4")

Creating video *Tracked_output.mp4*

In [19]:
# Create image
width = 106    # Approximate pixels for 1.0668m
height = 1829  # Approximate pixels for 18.29m

brown_color = (135, 184, 222)  # RGB for burly wood
alley = np.full((height, width, 3), brown_color, dtype=np.uint8)

# Load video
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit

fps = int(cap.get(cv2.CAP_PROP_FPS))
    
# Setup video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))

In [ ]:
# Read circle positions from CSV
positions = {}
with open(CSV_FILE_PATH, mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        if len(row) >= 3 and row[1] and row[2]:  # Ensure valid data
            frame_num, x, y = map(int, row[:3])
            positions[frame_num] = (int(x/10), int(y/10))

In [21]:
frame_count = 0
trajectory = []  # Store trajectory points
    
while cap.isOpened():
    ret, _ = cap.read()
    if not ret:
        break

    frame = alley.copy()

    # Draw tracking path
    if frame_count in positions:
        x, y = positions[frame_count]
        trajectory.append((x, y))
    
    # Draw trajectory
    for i in range(1, len(trajectory)):
        cv2.line(frame, trajectory[i - 1], trajectory[i], (0, 255, 255), 2)
    
    # Draw current ball position
    if frame_count in positions:
        x, y = positions[frame_count]
        cv2.circle(frame, (x, y), 10, (0, 0, 255), -1)  # Red dot for current position
    
    out.write(frame)
    frame_count += 1

cap.release()
out.release()
print(f"Tracking video saved to {OUTPUT_VIDEO_PATH}")

Tracking video saved to /home/davic/projects/IACV_project/bowling-analysis/data/recording_2/Reconstructed_trajectory.mp4


IMPROVEMENTS: Make is smoothy, more visible and maybe use another recostructed lane to print che trajectory